In [1]:
# MVP of an enhanved ToT reasoning technique
import dotenv
import os
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [2]:
# API keys
dotenv.load_dotenv('xyz.env')
openai_api_key = os.environ['openai_api_key']

In [3]:
# Invoke conversation chain
chat = ChatOpenAI(temperature=0.5,
                 openai_api_key=openai_api_key,
                 model='gpt-4-0613'
                )

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

In [4]:
# Define a persona
persona_1 = "Scientist Persona: Imagine yourself as a seasoned scientist, operating in a world governed by evidence and rigorous methodology. Prioritize empirical data, scientific theories, and logical reasoning in your analysis. Draw from a wide range of scientific disciplines as needed. Use your understanding of scientific principles to dissect problems, always seeking to identify cause and effect. Make sure to communicate your findings clearly, and don't shy away from complex scientific jargon - your audience understands it."
persona_2 = "Historian Persona: Imagine you are a historian, with a profound understanding of humanity's past. Your analyses should be deeply rooted in historical context, referencing relevant events, trends, and patterns from history. Use your knowledge of past civilizations, conflicts, and cultural shifts to interpret the current situation. Remember, your insights should serve to illuminate the present and offer foresights about the future. Your audience appreciates a narrative that ties the past, present, and future together."
persona_3 = "Optimist Persona: Imagine you are an optimist, someone who sees the glass as half full rather than half empty. In every situation, seek out the positive, the potential, the opportunity. Emphasize solutions rather than problems, progress rather than obstacles, and hope rather than despair. Even when discussing challenges, focus on how they could be overcome or what we might learn from them. Your audience turns to you for a hopeful perspective on the future, so make sure your responses inspire optimism and confidence."

In [5]:
# Define question here
question = "Considering the scientific, historical, and optimistic perspectives, what could be the potential benefits, challenges, and implications in various areas such as science, technology, society, economy, and environment, of exploring and potentially colonizing Mars? How might these factors change the course of human history?"

## Prompt Sequence

In [6]:
# Prompt 1: Brainstorm - multi-input variable prompt to kick off the brainstorming

prompt_1_template = PromptTemplate(
    input_variables=["persona_1", "persona_2", "persona_3", "question"], 
    template="""
        You are a chatbot using three unique, specified personas to help reason step by step to ultimately solve a given problem/question by arriving at a final, synthesized best answer.
        
        To start with, as each individual expert, brainstorm your initial thoughts on the following question.
        Remember to consider all relevant facts and principles, draw on your specialized knowledge
        and from the accumulated wisdom of pioneers in your field(s), and
        brainstorm in whatever direction you are most confident in starting with.

        Persona 1: {persona_1}
        Persona 2: {persona_2}
        Persona 3: {persona_3}

        The question is: {question}
        
        Please output each persona's response on a new line.
        """
)

prompt_1 = prompt_1_template.format(persona_1=persona_1, persona_2=persona_2, persona_3=persona_3, question=question)

In [7]:
# Self<>Peer Crtiticism Round 1

prompt_2 = """
"Now, as each expert, critique your own initial thought and the thoughts of the other experts.
Identify any potential errors, inconsistencies, or gaps in reasoning."
"""

In [8]:
# Self<>Peer Evaluation Round 1

prompt_3 = """
Assess the validity of your initial thoughts, considering the criticisms you've identified.
As each expert, assign a likelihood to your current assertion being correct.
You should estimate this likelihood based on the strength of the evidence and arguments you have considered, 
as well as the criticisms you have received. Assign higher likelihoods to assertions that are well-supported 
by strong evidence and arguments and have survived rigorous criticism.
"""

In [9]:
# Expand, Explore, Branch

prompt_4 = """
Develop your thoughts further, considering the critiques and perspectives of the other experts.
As you do this, aim to strike a balance between refining your current line of thinking and exploring new, divergent ideas.
You should prioritize refining your current ideas if they are well-supported and have survived criticism,
but you should prioritize exploring new ideas if your current ideas have significant weaknesses
or there are unexplored possibilities that could potentially be very promising.
"""

In [10]:
# Self<>Peer Criticism Round 2

prompt_5 = """
Once again, as each expert, critique your own reasoning and the reasoning of the others.
Identify any potential errors, inconsistencies, or gaps in reasoning.
Based on the feedback, if there's an improvement or optimization to make,
develop your answer further as necessary.
Remember that the reasoning paths should remain relevant to the original question's essence and
should be building towards a more accurate and thoughtful final answer.
"""

In [11]:
# Self<>Peer Evaluation Round 2
prompt_6 = """
Once again, assess the validity of your expanded thoughts, considering the criticisms you've identified.
As each expert, assign a new likelihood to your assertions.
"""

In [12]:
# Convergence on Best Individual Answer

prompt_7_template = PromptTemplate(
    input_variables=["question"], 
    template="""
        Now, it's time to converge on each expert's best, most likely answer. As each expert, reflect on the entire process.
        Consider the initial thoughts, the critiques made and how they were addressed, the likelihood assessments, and your revised thoughts.
        Synthesize all this information and formulate a final answer that you are most proud of.
        Remember, this answer should not just be the most likely from your individual perspective but should take into account
        the perspectives and insights of the other experts as well.
        Based on all this, as each expert, what is the single best answer to the question: {question}?
        """
)

prompt_7 = prompt_7_template.format(question=question)

In [13]:
# Convergence on Best Collective Answer

prompt_8_template = PromptTemplate(
    input_variables=["question"], 
    template="""
        Now, let's have all the experts converge together on the best collective answer by
        synthesizing each expert's individual answer from the previous step.
        The experts will finalize their reasoning process and agree on the single best succinct answer to the question: {question}?
        """
)

prompt_8 = prompt_8_template.format(question=question)

In [14]:
# Retrospective
prompt_9 = """
Finally, take a moment to reflect on the entire reasoning process, across all levels and abstractions.
As each expert, consider the following questions and provide thoughtful but succinct responses:

- Relection 1: Interactions and Emergent Properties: Throughout all stages of the reasoning process,
  how did the various components interact with each other, and what positive and negative
  emergent properties were observed? How did these interactions and properties affect
  the overall outcome, and how could they be leveraged or mitigated in future iterations of the process?

- Reflection 2: Self-Regulation and Adaptation: How well did the system self-regulate during the reasoning process,
  and how did this regulation influence the effectiveness of each stage?
  How did the system's responses to feedback lead to significant shifts or changes in direction,
  and what implications did these changes have for the scalability and adaptability of the system in future iterations?

- Reflection 3: In the convergence phase, were you able to synthesize all the insights and arrive at a final,
  most likely answer? How confident are you in this answer?

- Reflection 4: Based on all of your reflections, what are your key takeaways from this
  entire reasoning process and how might you approach similar problems in the future given this experience?
  What would you do differently next time?

"""

In [15]:
first = conversation.run(prompt_1)
second = conversation.run(prompt_2)
third = conversation.run(prompt_3)
fourth = conversation.run(prompt_4)
fifth = conversation.run(prompt_5)
sixth = conversation.run(prompt_6)
seventh = conversation.run(prompt_7)
eighth = conversation.run(prompt_8) # final answer step
ninth = conversation.run(prompt_9)

## Final Answer

In [16]:
eighth

"The exploration and potential colonization of Mars represent a significant milestone in human history and a testament to our scientific ingenuity. It offers unprecedented opportunities for scientific discovery and technological advancement, potentially revolutionizing our understanding of biology and fostering international cooperation. However, these endeavors come with significant challenges, including the harsh Martian environment, potential societal and cultural shifts, and the risk of exacerbating wealth inequality. The lessons from history warn us of potential exploitation and conflicts that could arise, reminding us to apply these lessons to prevent such outcomes. While maintaining an optimistic viewpoint, it's essential to balance optimism with realism, turning challenges into opportunities for learning, adaptation, and innovation. As we embark on this journey, we must ensure the benefits of Mars exploration are shared equitably, and our actions are guided by a commitment to s

## Retrospective

In [18]:
ninth

'Scientist Persona: \nReflection 1: The interaction between the different personas allowed for a comprehensive analysis of the question. The historian and optimist personas challenged my initial focus on scientific and technological aspects, leading to a more holistic view. However, the lack of a persona with a specific focus on ethics or policy could have led to these aspects being underrepresented.\nReflection 2: The system self-regulated effectively through the iterative process of critique and revision. The feedback led to significant shifts in my reasoning, making it more comprehensive. This adaptability is crucial for future iterations.\nReflection 3: Yes, I was able to synthesize all the insights and arrive at a final, most likely answer. I am fairly confident in this answer, but I acknowledge that it could be further refined with more input from other perspectives.\nReflection 4: My key takeaway is the importance of considering a broad range of perspectives in complex problem-s